### Generating point source distribution tables and density fields


In [ ]:
# mesh_y = 120
# mesh_x = int(wh_ratio * mesh_y)

# gr_x = np.linspace(alpha_min, alpha_max, mesh_x)
# gr_y = np.linspace(delta_min, delta_max, mesh_y)

# grid_x, grid_y = np.meshgrid(gr_x, gr_y)

# for slice_no in tqdm(range(0, 135)):

#     slice = Slice(slice_no)
#     sources = slice.sources
#     density = slice.density_excess

#     df = pd.DataFrame({'ID': sources.iloc[:, 6], 'RA': sources.iloc[:, 0], 'Dec': sources.iloc[:, 1], 'w': sources.iloc[:, 2]})
#     indices = np.where(np.isin(Data['ID'], df['ID']))[0]
#     last_nonZ = np.where(df['ID'] == 0)[0][0]

#     df['mass'] = np.zeros(len(df))
#     df['z'] = np.zeros(len(df))

#     df['mass'].iloc[0:last_nonZ] = Data[indices]['lp_mass_med']
#     df['z'].iloc[0:last_nonZ] = Data[indices]['lp_zBEST']

#     df['mass'].iloc[last_nonZ:] = np.mean(df['mass'].iloc[0:last_nonZ])
#     df['z'].iloc[last_nonZ:] = slice.center

#     df.to_csv('champ/sources_s=%d_z=%.3f.csv' %(slice_no, slice.center), index = False)

#     density_exc = slice.density_excess
#     np.savez('champ/densityExcess_s=%d_z=%.3f.npz' %(slice_no, slice.center), grid_x = grid_x, grid_y = grid_y, field = density_exc)

## Old


In [ ]:
# num_rows, num_cols = Slice(slice_no).density_contrast.shape

# ######################### Needs Correction #############################
# gr_x = np.linspace(alpha_min, alpha_max, num_cols)
# gr_y = np.linspace(delta_min, delta_max, num_rows)

# distance_scale_x = gr_x[1] - gr_x[0]
# distance_scale_y = gr_y[1] - gr_y[0]
# ######################### Needs Correction #############################

# def gaussian_kernel(x, y, x_prime, y_prime, L):
#     """
#     all inputs in the same unit (e.g., Mpc)
#     """
#     distance_squared = ((x - x_prime) * distance_scale_x)**2 + ((y - y_prime) * distance_scale_y)**2
#     exponent = -distance_squared / (2 * L**2)
#     return np.exp(exponent) / (2 * np.pi * L**2)

# def hessian_eigenvalues(field, L_Mpc, path = './eigenvalues'):
#     L = cosmo.arcsec_per_kpc_comoving(slice_centers[slice_no]).value * L_Mpc / 3.6
#     eigenvalues_field = np.empty((num_rows, num_cols), dtype=object)
    
#     for i in tqdm(range(0, num_rows)):
#         for j in range(num_cols):
#             hessian = np.zeros((2, 2))
#             for m in range(num_rows):
#                 for n in range(num_cols):
#                     weight = gaussian_kernel(i, j, m, n, L)
#                     hessian[0, 0] += weight * field[m, n] * (((m - i) * distance_scale_x)**2 - L**2) / (L**4)
#                     hessian[0, 1] += weight * field[m, n] * ((i - m) * (j - n) * distance_scale_x * distance_scale_y) / (L**4)
#                     hessian[1, 0] += weight * field[m, n] * ((i - m) * (j - n) * distance_scale_x * distance_scale_y) / (L**4)
#                     hessian[1, 1] += weight * field[m, n] * (((n - j) * distance_scale_y)**2 - L**2) / (L**4)

# ########################## Check this part ##########################
#             normalized_hessian = L * hessian                    # Check this
#             eigenvalues = np.linalg.eigvals(hessian)            # Check this
#             eigenvalues_field[i, j] = eigenvalues.tolist()
# ########################## Check this part ##########################
#     out_path_eigen = os.path.join(path, 'ev_slice%d_z=%.3f_L=%.2f Mpc.npy' %(slice_no, slice_centers[slice_no], L_Mpc))
#     np.save(out_path_eigen, eigenvalues_field)
#     return eigenvalues_field

In [ ]:
#### Modified Hessian
################ test ################
num_rows, num_cols = 120, 137
save_dir = './inputs/pre/diff'

gr_x = np.linspace(alpha_min, alpha_max, num_cols)
gr_y = np.linspace(delta_min, delta_max, num_rows)

distance_scale_x = gr_x[1] - gr_x[0]
distance_scale_y = gr_y[1] - gr_y[0]

def gaussian_kernel(x, y, x_prime, y_prime, L):
    distance_squared = ((x - x_prime) * distance_scale_x)**2 + ((y - y_prime) * distance_scale_y)**2
    exponent = -distance_squared / (2 * L**2)
    return np.exp(exponent) / (2 * np.pi * L**2)

def pre_hessian(slice, scales, box_size = 100, threshold_L = 0.05):
    redshift = slice_centers[slice]
    for L in scales:
        gaussian_kernel_path = os.path.join(save_dir, 'kernel_slice%d_L=%.2f Mpc.npy' %(slice, L))
        L_deg = cosmo.arcsec_per_kpc_comoving(redshift).value * L / 3.6

        if L_deg > threshold_L: 
            half_box_size = box_size / 2
        else:
            half_box_size = box_size / 4

        # Kernel tensor
        if not os.path.exists(gaussian_kernel_path):
            print('Calculating kernel tensor for L = %.2f Mpc at slice %d' %(L, slice))
            gauss_tensor = np.zeros((num_rows, num_cols, num_rows, num_cols))
            for i in tqdm(range(0, num_rows)):
                for j in range(num_cols):
                    min_row = int(max(0, i - half_box_size))
                    max_row = int(min(num_rows, i + half_box_size + 1))
                    min_col = int(max(0, j - half_box_size))
                    max_col = int(min(num_cols, j + half_box_size + 1))
                    for m in range(min_row, max_row):
                        for n in range(min_col, max_col):
                            gauss_tensor[i, j, m, n] = gaussian_kernel(i, j, m, n, L_deg)
            np.save(os.path.join(save_dir, 'kernel_slice%d_L=%.2f Mpc.npy' %(slice, L)), gauss_tensor)

        # Shape tensor
        shape_path = os.path.join(save_dir, 'shape_slice%d_L=%.2f Mpc.npy' %(slice, L))
        if not os.path.exists(shape_path):
            print('Calculating shape tensor for L = %.2f Mpc at slice %d' %(L, slice))
            shape_tensor = np.zeros((num_rows, num_cols, num_rows, num_cols, 2, 2))
            for i in tqdm(range(0, num_rows)):
                for j in range(num_cols):
                    min_row = int(max(0, i - half_box_size))
                    max_row = int(min(num_rows, i + half_box_size + 1))
                    min_col = int(max(0, j - half_box_size))
                    max_col = int(min(num_cols, j + half_box_size + 1))
                    for m in range(min_row, max_row):
                        for n in range(min_col, max_col):
                            sub_shape = np.zeros((2, 2))

                            sub_shape[0, 0] = (((m - i) * distance_scale_x)**2 - L_deg**2) / (L_deg**4)
                            sub_shape[0, 1] = ((i - m) * (j - n) * distance_scale_x * distance_scale_y) / (L_deg**4)
                            sub_shape[1, 0] = ((i - m) * (j - n) * distance_scale_x * distance_scale_y) / (L_deg**4)
                            sub_shape[1, 1] = (((n - j) * distance_scale_y)**2 - L_deg**2) / (L_deg**4)

                            shape_tensor[i, j, m, n] = sub_shape
            np.save(os.path.join(save_dir, 'shape_slice%d_L=%.2f Mpc.npy' %(slice, L)), shape_tensor) 